In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

pcam, pcam_info = tfds.load('patch_camelyon', with_info=True,
                            data_dir='/content/drive/My Drive/Colab Notebooks')

In [0]:
# Import NumPy to handle array's and Matplotlib for plotting loss curves
import numpy as np
import matplotlib.pyplot as plt

# Import TensorFlow and relevant Keras classes to setup the model
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
stacked_models_names = ["geert", "densenet"]

stacked_models = [tf.keras.models.load_model("/content/drive/My Drive/Saved/" + model) for model in stacked_models_names]

In [5]:
#First setup the input to the network which has the dimensions of the patches contained within PatchCAMELYON
input_shape = Input(shape=(1,2 * len(stacked_models)))

# Now we define the layers of the convolutional network: three blocks of two convolutional layers and a max-pool layer.
#x = Dense(16, activation='relu')(input_shape)

x = Flatten()(input_shape)

predictions = Dense(16, activation='relu')(x)

predictions = Dense(2, activation='softmax')(x)

# Now we define the inputs/outputs of the model and setup the optimizer. In this case we use regular stochastic gradient
# descent with Nesterov momentum. The loss we use is cross-entropy and we would like to output accuracy as an additional metric.
model = Model(inputs=input_shape, outputs=predictions)
sgd_opt = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=True)
model.compile(optimizer= sgd_opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1, 4)]            0         
_________________________________________________________________
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [0]:
import random
def convert_sample(sample):
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.rot90(image, random.randint(0,3))
    image = tf.image.random_flip_left_right(image)
    #noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=(.1), dtype=tf.float32) 
    #image = image + noise
    image = tf.expand_dims(image,0)

    output =  tf.concat([m(image) for m in stacked_models],1)
   

    label = tf.one_hot(label, 2, dtype=tf.float32)
    return output, label

In [0]:
train_pipeline = pcam['train'].map(convert_sample,
                                   num_parallel_calls=8).shuffle(1024).repeat().batch(64).prefetch(2)
valid_pipeline = pcam['validation'].map(convert_sample,
                                        num_parallel_calls=8).repeat().batch(128).prefetch(2)

In [0]:
hist = model.fit(train_pipeline,
                 validation_data=valid_pipeline,
                 verbose=2, epochs=5, steps_per_epoch=200, validation_steps=256)

Epoch 1/5
200/200 - 2744s - loss: 0.8162 - accuracy: 0.5157 - val_loss: 0.6406 - val_accuracy: 0.7567
Epoch 2/5
200/200 - 2778s - loss: 0.5551 - accuracy: 0.8323 - val_loss: 0.5167 - val_accuracy: 0.8145
Epoch 3/5


In [0]:
test_pipeline = pcam['test'].map(convert_sample, num_parallel_calls=8).batch(128).prefetch(2)
print("Test set accuracy is {0:.4f}".format(model.evaluate(test_pipeline, steps=32768//128, verbose=0)[1]))